### A Simple Data Pull From Twitter's REST API

In [41]:
import oauth2
from credentials import *

In [42]:
# assigning key variables
API_KEY = api_key
API_SECRET = api_secret
TOKEN_KEY = access_token
TOKEN_SECRET = access_token_secret

In [46]:
# function to create OAuth connection
def oauth_req(url, key, secret, http_method='GET', post_body="", http_headers=None):
    consumer = oauth2.Consumer(key=API_KEY, secret=API_SECRET)
    token = oauth2.Token(key=key, secret=secret)
    client = oauth2.Client(consumer, token)
    
    # Encode the post_body if it's a string
    if isinstance(post_body, str):
        post_body = post_body.encode('utf-8')
        
    resp, content = client.request(url, method=http_method, body=post_body, headers=http_headers)
    return content

url = 'https://api.twitter.com/1.1/search/tweets.json?q=%23childlabor'
data = oauth_req(url, TOKEN_KEY, TOKEN_SECRET)
print(data)

b'{"errors":[{"message":"You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product","code":453}]}\n'
